In [1]:
import os
import gc
import numpy as np
import pandas as pd
import JS_NV_crawler as JS

# 데이터 로딩

In [227]:
# 네이버 서울 음식점
nv_place = pd.read_excel(os.path.join("..","..", "data","JS_0601_select_place.xlsx"))

In [174]:
# 트립어드바이저 서울 음식점
ta_place = pd.read_excel(os.path.join("..","..", "data","TA_음식점_한글이름_주소_정보.xlsx"))

In [7]:
nv_place.shape

(9867, 18)

In [8]:
ta_place.shape

(5574, 7)

In [67]:
# 이름만으로 매칭되는 것은 900개 정도
nv_place.merge(ta_place, left_on = 'place.name', right_on = 'name', how = 'inner')

In [109]:
# 주소로만 매칭하는 것도 안될 것 같음, 심지어 위경도도 같아버림
nv_place.merge(ta_place, on = 'land.addr', how = 'inner').head(5)[['place.name', 'name', 'land.addr', 'lat_x', 'lng_x', 'lat_y', 'lng_y']]

,place.name,name,land.addr,lat_x,lng_x,lat_y,lng_y
0,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
1,안동찜닭&해물떡찜,롯데리아,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
2,안동찜닭&해물떡찜,테이블스타,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
3,안동찜닭&해물떡찜,하동관 코엑스몰직영점,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427
4,안동찜닭&해물떡찜,테라로사,서울특별시 강남구 삼성동 159 코엑스,37.511662,127.059427,37.511662,127.059427


# ta 위경도 값 없는 것 가져오기

In [71]:
addrs = ta_place['address']

In [72]:
from tqdm import tqdm

In [73]:
addr_list = []
for addr in tqdm(addrs):
    value = JS.NVgeocoding(addr)
    addr_list.append(value)

100%|██████████| 5574/5574 [08:57<00:00, 10.37it/s] 


# 트립어드바이저 새로운 주소 붙여주기

In [35]:
temp = pd.DataFrame(addr_list, columns =['land.addr', 'road.addr', 'lat', 'lng'])

In [175]:
ta_place = pd.concat([ta_place, temp], axis = 1)

In [205]:
# 트립어드바이저 서울 음식점 위치 네이버로 업데이트
ta_place.to_excel(os.path.join("..","..", "data","TA_음식점_한글이름_주소_정보.xlsx"))

# nv 와 ta 거리 계산하기 9000 * 5000

In [93]:
from scipy.spatial import distance

In [110]:
nv_place['lat'] = nv_place['lat'].astype(float)
nv_place['lng'] = nv_place['lng'].astype(float)
ta_place['lat'] = ta_place['lat'].astype(float)
ta_place['lng'] = ta_place['lng'].astype(float)

In [111]:
from tqdm import tqdm

In [112]:
i = 0
total = []
for args in tqdm(nv_place[['place.id','lat', 'lng']].values):
    nv_idx = args[0]
    n_lat, n_lng = args[1:]
    dis_all = []
    for kwargs in ta_place[['location_id', 'lat', 'lng']].values:
        ta_idx = kwargs[0]
        t_lat, t_lng = kwargs[1:]
        dis = distance.euclidean((n_lat, n_lng), (t_lat, t_lng))
        value = [nv_idx, ta_idx, dis]
        dis_all.append(value)
        
    if float(min(np.array(dis_all)[:,2])) < 0.0001:
        loc = np.argmin(np.array(dis_all)[:,2])
        value = np.array(dis_all)[loc]
    else:
        value = 'none'
    i += 1
    total.append(value)

100%|██████████| 9867/9867 [16:27<00:00,  9.99it/s]


In [113]:
len(total)

9867

In [114]:
total = pd.DataFrame(total)

In [139]:
total['is_valid'] = total[0].apply(lambda x: 1 if len(str(x).split(',')[0]) > 4 else 0)

In [141]:
total = total.loc[total['is_valid'] == 1]

In [161]:
total['place.id'] = total[0].apply(lambda x : x[0])
total['location_id'] = total[0].apply(lambda x : x[1])

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [163]:
total.drop([0,'is_valid'], axis = 1, inplace = True)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [164]:
total

,place.id,location_id
14,33879152,9141260.0
34,36166539,3938485.0
62,11605455,2241571.0
81,1266909878,9318901.0
84,1288556612,3911442.0
...,...,...
9839,35162551,9029313.0
9840,36667369,8942305.0
9843,37204578,4034834.0
9844,38404658,9402393.0


In [159]:
def matchName(nv, ta):
    nv_name = nv_place.loc[nv_place['place.id'] == nv, 'place.name'].values[0]
    ta_name = ta_place.loc[ta_place['location_id'] == float(ta), 'name'].values[0]
    return nv_name, ta_name

In [165]:
matchName(*total.values[:,:2][0])

('부산아구', '첨벙아구찜')

In [166]:
total.to_excel(os.path.join("..","..", "data","JS_09_between_distance.xlsx"))

# idx 기준 주소와 이름 가져오기

In [180]:
dt = total.copy()

In [177]:
nv_place_t = nv_place[['place.id', 'place.name', 'land.addr']]

In [178]:
ta_place_t = ta_place[['location_id', 'name', 'land.addr']]

In [181]:
dt = dt.merge(nv_place_t, on = 'place.id', how = 'left')

In [182]:
dt['location_id']

0        9141260.0
1        3938485.0
2        2241571.0
3        9318901.0
4        3911442.0
           ...    
1063     9029313.0
1064     8942305.0
1065     4034834.0
1066     9402393.0
1067    13964808.0
Name: location_id, Length: 1068, dtype: object

In [183]:
dt['location_id'] = dt['location_id'].apply(lambda x : int(float(x)) if x != 'o' else x)

In [184]:
dt = dt.merge(ta_place_t, on = 'location_id', how = 'left')

In [185]:
dt

,place.id,location_id,place.name,land.addr_x,name,land.addr_y
0,33879152,9141260,안동찜닭&해물떡찜,서울특별시 강남구 삼성동 159 코엑스,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스
1,36166539,3938485,에스키모 코엑스몰점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울
2,11605455,2241571,가미분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트
3,1266909878,9318901,우리두리,서울특별시 용산구 동자동 43-205 한국철도공사,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사
4,1288556612,3911442,마포만두,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,스타벅스 오목교점,서울특별시 양천구 목동 404-177 썬택씨티빌딩
...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역
1064,36667369,8942305,계절밥상 동대문롯데피트인점,서울특별시 중구 을지로6가 21-31 롯데 FiTin,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin
1065,37204578,4034834,계절밥상 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드
1066,38404658,9402393,계절밥상 서울역사점,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사


In [187]:
dt = dt[['place.id', 'location_id', 'place.name', 'name', 'land.addr_x', 'land.addr_y']]

In [188]:
dt

,place.id,location_id,place.name,name,land.addr_x,land.addr_y
0,33879152,9141260,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,서울특별시 강남구 삼성동 159 코엑스
1,36166539,3938485,에스키모 코엑스몰점,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트
3,1266909878,9318901,우리두리,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사,서울특별시 용산구 동자동 43-205 한국철도공사
4,1288556612,3911442,마포만두,스타벅스 오목교점,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,서울특별시 양천구 목동 404-177 썬택씨티빌딩
...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,서울특별시 용산구 한강로3가 40-999 용산역
1064,36667369,8942305,계절밥상 동대문롯데피트인점,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin,서울특별시 중구 을지로6가 21-31 롯데 FiTin
1065,37204578,4034834,계절밥상 잠실점,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,서울특별시 송파구 신천동 7-18 롯데캐슬골드
1066,38404658,9402393,계절밥상 서울역사점,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사


# 서로서로 이름이 속하면 같은 매장으로

In [189]:
arg = dt.values[0]

In [191]:
arg[2], arg[3]

('안동찜닭&해물떡찜', '커피빈 삼성오크우드점')

In [192]:
arg[3].replace(' ','')

'커피빈삼성오크우드점'

In [201]:
name_duplicate = []
for arg in dt.values:
    nv_name = arg[2].replace(' ', '')
    ta_name = arg[3].replace(' ', '')
    if nv_name in ta_name:
        value = 1
    elif ta_name in nv_name:
        value = 1
    else:
        value = 0
    name_duplicate.append(value)

In [202]:
dt['name_duplicate'] = name_duplicate

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [203]:
dt

,place.id,location_id,place.name,name,land.addr_x,land.addr_y,name_duplicate
0,33879152,9141260,안동찜닭&해물떡찜,커피빈 삼성오크우드점,서울특별시 강남구 삼성동 159 코엑스,서울특별시 강남구 삼성동 159 코엑스,0
1,36166539,3938485,에스키모 코엑스몰점,아웃백 코엑스점,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,서울특별시 강남구 삼성동 159 코엑스인터콘티넨탈서울,0
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,1
3,1266909878,9318901,우리두리,파리크라상,서울특별시 용산구 동자동 43-205 한국철도공사,서울특별시 용산구 동자동 43-205 한국철도공사,0
4,1288556612,3911442,마포만두,스타벅스 오목교점,서울특별시 양천구 목1동 404-177 썬택씨티빌딩,서울특별시 양천구 목동 404-177 썬택씨티빌딩,0
...,...,...,...,...,...,...,...
1063,35162551,9029313,계절밥상 용산아이파크몰점,계절밥상 - 용산아이파크몰점,서울특별시 용산구 한강로3가 40-999 용산역,서울특별시 용산구 한강로3가 40-999 용산역,0
1064,36667369,8942305,계절밥상 동대문롯데피트인점,엔제리너스,서울특별시 중구 을지로6가 21-31 롯데 FiTin,서울특별시 중구 을지로6가 21-31 롯데 FiTin,0
1065,37204578,4034834,계절밥상 잠실점,무스쿠스 잠실점,서울특별시 송파구 신천동 7-18 롯데캐슬골드,서울특별시 송파구 신천동 7-18 롯데캐슬골드,0
1066,38404658,9402393,계절밥상 서울역사점,맥도날드,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,서울특별시 중구 봉래동2가 122-11 경부고속철도서울민자역사,0


In [209]:
dt.loc[dt['name_duplicate'] == 1]

,place.id,location_id,place.name,name,land.addr_x,land.addr_y,name_duplicate
2,11605455,2241571,가미분식,가미 분식,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,1
22,10976694,5926853,유래회관,유래회관,서울특별시 성동구 홍익동 357 유래회관,서울특별시 성동구 홍익동 357 유래회관,1
24,11398664,4032150,대성갈비,대성갈비,서울특별시 성동구 성수동1가 668-21,서울특별시 성동구 성수동1가 668-21,1
25,11663081,3922609,철길왕갈비살,철길 왕 갈비살,서울특별시 마포구 창전동 4-2,서울특별시 마포구 창전동 4-2,1
26,11677544,4033804,황소고집,황소고집,서울특별시 종로구 관철동 11-11,서울특별시 종로구 관철동 11-11,1
...,...,...,...,...,...,...,...
1029,12447840,4070735,김영모과자점 도곡타워점,김영모과자점 도곡타워점,서울특별시 강남구 도곡동 467-19 현대비전21,서울특별시 강남구 도곡2동 467-19 현대비전21,1
1030,13575110,6818439,미즈컨테이너 강남 웨스트,미즈컨테이너,서울특별시 서초구 서초동 1316-29 타임빌딩,서울특별시 서초구 서초동 1316-29 타임빌딩,1
1032,32817375,4784018,생어거스틴 경복궁점,생어거스틴 경복궁점,서울특별시 종로구 중학동 19 더케이트윈타워,서울특별시 종로구 중학동 19 더케이트윈타워,1
1039,33758018,8948116,아웃닭 홍대점,아웃닭,서울특별시 마포구 서교동 409-17,서울특별시 마포구 서교동 409-17,1


In [207]:
dt.loc[dt['name_duplicate'] == 0].to_excel('./place_check.xlsx')

In [210]:
dt2 = pd.read_excel('./place_check.xlsx')

In [215]:
dt.loc[dt['name_duplicate'] == 0, 'name_duplicate'] = dt2['매칭여부'].values

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [230]:
# 네이버와 트립어드바이저 같은 place
same_place = dt.loc[dt['name_duplicate'] == 1, ['place.id', 'location_id']]

In [233]:
same_place.to_excel(os.path.join("..", "..", "data", "JS_0602_same_place_index.xlsx"))